In [ ]:
# Obter a varia

<h1 style='font-size:40px'> Stock Price Variation Predictor</h1>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            The present project aims to create a stock price variation predictor using the corporations' fundamentals. 
        </li>
        <li> 
            The model must estimate the given quarter's 80th percentile variation using the financial statements from the previous quarter.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Building the dataset</h2>
<div> 
    <ul style='font-size:20px'> 
        <li> 
            Unlike most of the ML projects out there, in this one we'll face the challenge of making the dataset with which our models will carry out their predictions. Fortunately, the yfinance library can swiftly retrieve the corporate financial statements needed.
        </li>
    </ul>
</div>

In [142]:
from yfinance import Ticker
petr3 = Ticker('PETR3.SA')
pd.set_option('max_columns', None)

In [143]:
from yfinance import Ticker
petr3_quarterly_balance_sheet = petr3.quarterly_balance_sheet.T
petr3_quarterly_balance_sheet.index = pd.PeriodIndex(petr3_quarterly_balance_sheet.index, freq='Q')

petr3_quarterly_financials = petr3.quarterly_financials.T
petr3_quarterly_financials.index = pd.PeriodIndex(petr3_quarterly_financials.index, freq='Q')

petr3_quarterly_cashflow = petr3.quarterly_cashflow.T
petr3_quarterly_cashflow.index = pd.PeriodIndex(petr3_quarterly_cashflow.index, freq='Q')

petr3_quarterly_earnings = petr3.quarterly_earnings
petr3_quarterly_earnings.index = pd.PeriodIndex(petr3_quarterly_earnings.index, freq='Q')

In [128]:
(petr3_quarterly_cashflow
     .merge(petr3_quarterly_earnings, left_index=True, right_index=True)
     .merge(petr3_quarterly_financials, left_index=True, right_index=True)
     .merge(petr3_quarterly_balance_sheet, left_index=True, right_index=True))

,Investments,Change To Liabilities,Total Cashflows From Investing Activities,Net Borrowings,Total Cash From Financing Activities,Change To Operating Activities,Net Income_x,Change In Cash,Repurchase Of Stock,Effect Of Exchange Rate,Total Cash From Operating Activities,Depreciation,Change To Inventory,Change To Account Receivables,Other Cashflows From Financing Activities,Change To Netincome,Capital Expenditures,Issuance Of Stock,Revenue,Earnings,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest_x,Net Income_y,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,Interest Expense,Extraordinary Items,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares,Intangible Assets,Total Liab,Total Stockholder Equity,Minority Interest_y,Other Current Liab,Total Assets,Common Stock,Other Current Assets,Retained Earnings,Other Liab,Good Will,Treasury Stock,Other Assets,Cash,Total Current Liabilities,Deferred Long Term Asset Charges,Short Long Term Debt,Other Stockholder Equity,Property Plant Equipment,Total Current Assets,Long Term Investments,Net Tangible Assets,Short Term Investments,Net Receivables,Long Term Debt,Inventory,Accounts Payable
2022Q1,-1.913069e+09,777528000.0,-2.362861e+09,5.232984e+09,4.798916e+09,-2.133266e+09,1.087620e+08,3.199996e+09,-77270000.0,-4.410040e+08,1.204945e+09,400986000.0,-214288000.0,7.352350e+08,-3.567980e+08,1.050161e+09,-448759000.0,NaN,22340812000,108762000,None,None,1255045000.0,1750323000.0,108762000.0,1132440000.0,3482105000.0,2327584000.0,2327584000.0,22081000.0,-547818000.0,None,None,None,762642000.0,22340812000.0,20013228000.0,18858707000.0,-1072539000.0,None,492403000.0,108762000.0,5.034958e+09,4.345171e+10,3.359717e+09,1.750323e+09,4.057520e+09,4.856175e+10,8.204391e+09,801032000.0,2.871645e+09,1.085747e+09,1.603877e+09,-7.716319e+09,4.299771e+09,4.959478e+09,1.638612e+10,5.783190e+08,7.321161e+09,-6.788174e+09,9.159635e+09,2.118036e+10,7.283149e+09,-3.279118e+09,6.105469e+09,5.170527e+09,2.556545e+10,4.037162e+09,3.924174e+09
2021Q4,-3.694600e+07,484815000.0,-7.834100e+08,-1.604800e+08,-2.010490e+09,2.275160e+08,6.497460e+08,-1.219739e+09,-507875000.0,9.105700e+07,1.483104e+09,428328000.0,-115426000.0,-8.242040e+08,-1.342135e+09,1.309895e+09,-745897000.0,NaN,23941175000,649746000,None,None,1895054000.0,1654803000.0,649746000.0,1276392000.0,5026841000.0,3661066000.0,3661066000.0,89383000.0,-497079000.0,None,None,None,573533000.0,23941175000.0,20280109000.0,18914334000.0,-1766012000.0,None,1321521000.0,649746000.0,6.038516e+09,4.243154e+10,3.916920e+09,1.654803e+09,3.708157e+09,4.800326e+10,8.204391e+09,761155000.0,2.762558e+09,1.217895e+09,1.892630e+09,-7.050029e+09,4.441181e+09,1.759482e+09,1.704766e+10,8.850480e+08,6.977250e+09,-6.199154e+09,9.792535e+09,1.951430e+10,6.324091e+09,-4.014226e+09,6.337834e+09,6.195722e+09,2.368455e+10,4.351282e+09,3.826714e+09
2021Q3,-2.500649e+09,107943000.0,-3.031197e+09,-1.246959e+09,-2.988142e+09,4.045200e+08,1.675243e+09,-8.090740e+08,-507875000.0,4.883200e+08,4.721945e+09,384488000.0,-181802000.0,4.261730e+08,-1.742281e+09,2.321739e+09,-529269000.0,1098000.0,23637794000,1675243000,None,None,2995853000.0,1806895000.0,1675243000.0,1067143000.0,5415148000.0,4271895000.0,4271895000.0,76110000.0,-440000000.0,None,None,None,553730000.0,23637794000.0,19365899000.0,18222646000.0,-1276042000.0,None,2442123000.0,1675243000.0,6.043508e+09,4.043598e+10,4.866932e+09,1.806895e+09,3.823644e+09,4.710981e+10,8.204391e+09,362726000.0,2.942597e+09,1.201118e+09,1.844832e+09,-6.280056e+09,4.580636e+09,2.979221e+09,1.654516e+10,1.012034e+09,7.396894e+09,-5.937056e+09,9.076898e+09,2.548886e+10,7.507700e+07,-3.021408e+09,1.255755e+10,5.287198e+09,2.221290e+10,4.224106e+09,3.097978e+09
2021Q2,-1.888400e+07,622833000.0,-4.544780e+08,1.743714e+09

In [141]:
import pandas_datareader as web
web.DataReader('CIEL3.SA', 'yahoo', start='04-01-2021').resample('Q')['Close'].agg()

Date
2021-06-30    4.38
2021-09-30    3.65
2021-12-31    2.64
2022-03-31    3.11
2022-06-30    4.04
Freq: Q-DEC, Name: Close, dtype: float64

In [121]:
petr3_quarterly_balance_sheet.join([petr3_quarterly_financials, petr3_quarterly_cashflow, petr3_quarterly_earnings], lsuffix='L_', rsuffix='R_')

ValueError: Indexes have overlapping values: Index(['Minority Interest', 'Net Income'], dtype='object')